In [200]:
#pip install opencv-python mediapipe msvc-runtime
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

import time
import ast

import cv2
import random
import numpy as np
import matplotlib.pyplot as plt


from google.protobuf.internal.containers import RepeatedCompositeFieldContainer

In [201]:
file_path_to_write = './results/data_pose.txt'

In [202]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
	min_detection_confidence=0.75,
	min_tracking_confidence=0.5
)


# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

drawing_spec_lines_r = mp_drawing.DrawingSpec(  # линии соединения
          color=(255,0,0),
          thickness=2,
          circle_radius=1
        )

drawing_spec_lines_l = mp_drawing.DrawingSpec(  # линии соединения
          color=(0,255,0),
          thickness=2,
          circle_radius=1
        )


image = cv2.imread(r"C:\Users\ivano\Desktop\SBER\demo2.jpg")

# Converting the from BGR to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Making predictions using holistic model
# To improve performance, optionally mark the image as not writeable to
# pass by reference.
image.flags.writeable = False
results = holistic_model.process(image)
image.flags.writeable = True

# # Create a black image
image = np.zeros(image.shape, np.uint8) 
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# Drawing the Facial Landmarks
mp_drawing.draw_landmarks(
  image,
  results.face_landmarks,
  mp_holistic.FACEMESH_CONTOURS,
  mp_drawing.DrawingSpec(
    color=(255,0,255),
    thickness=1,
    circle_radius=1
  ),
  mp_drawing.DrawingSpec(
    color=(0,255,255),
    thickness=1,
    circle_radius=1
  )
)

# Drawing Right hand Land Marks
mp_drawing.draw_landmarks(
  image, 
  results.right_hand_landmarks, 
  mp_holistic.HAND_CONNECTIONS,
  connection_drawing_spec=drawing_spec_lines_r
)

# Drawing Left hand Land Marks
mp_drawing.draw_landmarks(
  image, 
  results.left_hand_landmarks, 
  mp_holistic.HAND_CONNECTIONS,
  connection_drawing_spec=drawing_spec_lines_l
)

# Drawing pose Marks
mp_drawing.draw_landmarks(
  image, 
  results.pose_landmarks, 
  mp_holistic.POSE_CONNECTIONS
)

In [203]:
def record_points_file(landmarks_list=results.pose_landmarks.landmark, file_name=file_path_to_write):
    """
    Записываем результаты в файл для добавления в словарь
    """
    data_to_write = []
    for landmark in landmarks_list:
        data_to_write.append({
            'x': landmark.x,
            'y': landmark.y,
            'z': landmark.z,
            'visibility': landmark.visibility
        })

    print('Запись файла...')
    with open(file_path_to_write, 'w') as file:
        file.write(str(data_to_write))
    print('Успешно!')

    return data_to_write

### Адаптируем results к потоку фоток(цикл + размерность+=1) 

### Дальше этот `results` отдаем на сглаживание изображений

In [204]:
def read_points_file(file=file_path_to_write, image=image):
    """
    Чтение точек из файла или объекта для отрисовки
    + отрисовка тестовая
    На вход: путь или объект
    На выход: заисанный объект в переменную
    """
    if type(file) is str:

        with open(file, 'r') as f:
            content = f.read()

        data = ast.literal_eval(content)
    else:
        data = file


    landmark_list = landmark_pb2.NormalizedLandmarkList()

    print('Читаем точки..')
    for point in data:
    # Добвление NormalizedLandmark в NormalizedLandmarkList
        landmark = landmark_list.landmark.add()
        landmark.x = point['x'] 
        landmark.y = point['y']
        landmark.z = point['z']
        landmark.visibility = point['visibility']
    print('Точки прочитаны успешно!\n')
    
    print('Создание класса визуализации...')
    # # Теперь вы можете использовать pose_landmarks с функцией draw_landmarks
    mp_drawing.draw_landmarks(
        image, 
        landmark_list, 
        mp_holistic.POSE_CONNECTIONS
    )
    print('Класс создан без ошибок!\n')

    print('Вывод изображения...')
    cv2.imshow("Facial and Hand Landmarks", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Работа завершена!')

    return image

In [205]:
results = record_points_file()
image_to_draw = read_points_file(results, image)

Запись файла...
Успешно!
Читаем точки..
Точки прочитаны успешно!

Создание класса визуализации...
Класс создан без ошибок!

Вывод изображения...
Работа завершена!
